In [19]:
from datasets import load_dataset
import random
random.seed(42)


In [20]:
def remove_random_word_in_string(s: str) -> str:

    words = s.split()
    if not words: 
        return s

    idx = random.randrange(len(words))
    target_word = words[idx]    
    words.pop(idx)
    print(f'the word {target_word} is removed')
    return " ".join(words)

def add_random_word_in_string(s: str) -> str:

    words = s.split()
    if not words: 
        return s

    idx = random.randrange(len(words))
    target_word = words[idx]
    words.insert(idx, words[idx])
    # print(f'the word {target_word} is repeated')
    return " ".join(words)

In [18]:

dataset = load_dataset("Maxwell-Jia/AIME_2024")

dataset_remove_one = dataset.map(lambda example: {"Problem": remove_random_word_in_string(example["Problem"])})
dataset_add_one = dataset.map(lambda example: {"Problem": add_random_word_in_string(example["Problem"])})

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

the word following is repeat
the word from is remove
the word her is repeat
the word the is remove
the word two is remove
the word (2 is repeat
the word so is remove
the word Find is remove
the word 1, is remove
the word of is repeat
the word \[(75+117i)z is remove
the word top is remove
the word hours, is repeat
the word changed is repeat
the word to is remove
the word for is remove
the word $r$ is remove
the word a is remove
the word $AI=IM=MI_{a}=\sqrt{156}$ is remove
the word all is remove
the word these is remove
the word 200, is repeat
the word $A$, is repeat
the word median is repeat
the word these is repeat
the word to is remove
the word in is repeat
the word tetrahedron is remove
the word GHD) is remove
the word divided is remove


In [7]:
dataset["train"]['Problem'][0]

'Let $x,y$ and $z$ be positive real numbers that satisfy the following system of equations: \n\\[\\log_2\\left({x \\over yz}\\right) = {1 \\over 2}\\]\n\\[\\log_2\\left({y \\over xz}\\right) = {1 \\over 3}\\]\n\\[\\log_2\\left({z \\over xy}\\right) = {1 \\over 4}\\]\nThen the value of $\\left|\\log_2(x^4y^3z^2)\\right|$ is $\\tfrac{m}{n}$ where $m$ and $n$ are relatively prime positive integers. Find $m+n$.'